Import a bunch of stuff

In [112]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import numpy as np
import pandas as pd
import re
import time
import csv
import os
import json
import urllib
import pickle
import pymongo
from pymongo import MongoClient
import pprint
from bson import json_util
import json
import pickle
import pprint

Open mongod db: Capstone, collection = project_id_returns

In [3]:
client = MongoClient()
db = client.capstone_project
collection = db.product_id_returns
# print collection statistics
# print (db.command("collstats"))

# print database statistics
print (db.command("dbstats"))

{'db': 'capstone_project', 'collections': 2, 'views': 0, 'objects': 90905, 'avgObjSize': 5084.188900500522, 'dataSize': 462178192.0, 'storageSize': 164089856.0, 'numExtents': 0, 'indexes': 2, 'indexSize': 946176.0, 'fsUsedSize': 83982340096.0, 'fsTotalSize': 499963170816.0, 'ok': 1.0}


Example data for one item

In [5]:
a = collection.find_one({'itemId': 34953581})
a

{'_id': ObjectId('5a514a0095cdbc13b4a60726'),
 'addToCartUrl': 'http://c.affil.walmart.com/t/api01?l=http%3A%2F%2Faffil.walmart.com%2Fcart%2FaddToCart%3Fitems%3D34953581%7C1%26affp1%3DEyJ2pD5bvQhSw0Bdth4-NPmOX17ATelCjl1M0_dPIfo%26affilsrc%3Dapi%26veh%3Daff%26wmlspartner%3Dreadonlyapi',
 'affiliateAddToCartUrl': 'http://linksynergy.walmart.com/fs-bin/click?id=|LSNID|&offerid=223073.7200&type=14&catid=8&subid=0&hid=7200&tmpid=1082&RD_PARM1=http%253A%252F%252Faffil.walmart.com%252Fcart%252FaddToCart%253Fitems%253D34953581%257C1%2526affp1%253DEyJ2pD5bvQhSw0Bdth4-NPmOX17ATelCjl1M0_dPIfo%2526affilsrc%253Dapi',
 'attributes': {'actualColor': 'Charcoal',
  'color': 'Gray',
  'finerCateg': 'Armchairs'},
 'availableOnline': False,
 'brandName': '10 Spring Street',
 'bundle': False,
 'categoryNode': '4044_103150_4038',
 'categoryPath': 'Home/Furniture/Living Room Furniture/Accent Chairs',
 'clearance': False,
 'color': 'Gray',
 'customerRating': '4.043',
 'customerRatingImage': 'http://i2.walmart

Sample Query:

In [18]:
results = db.product_id_returns.find({'$or': [{'availableOnline' : False}, {'availableOnline' : True}]},
                                    {'itemId': 1,
                                     'shortDescription' : 1,
                                     'imageEntities.thumbnailImage' : 1,
                                     'categoryNode' : 1,
                                     'categoryPath' : 1,
                                     'name' : 1})
results[4]

{'_id': ObjectId('5a514a0095cdbc13b4a60728'),
 'categoryNode': '4044_103150_4037',
 'categoryPath': 'Home/Furniture/Kitchen & Dining Furniture/Pantries',
 'imageEntities': [{'thumbnailImage': 'https://i5.walmartimages.com/asr/662d67cb-d3ad-4df6-9f62-3f8516b177b8_1.1eb4ece9dee5ed505973a090d5a312c4.jpeg?odnHeight=100&odnWidth=100&odnBg=FFFFFF'},
  {'thumbnailImage': 'https://i5.walmartimages.com/asr/393a74b0-6215-4816-939f-671bb4bd84f7_1.bfcb5aa546597a89950d01114fed9371.jpeg?odnHeight=100&odnWidth=100&odnBg=FFFFFF'}],
 'itemId': 35333696,
 'name': 'Crosley Furniture Parsons Pantry',
 'shortDescription': 'Add function and style to your kitchen with the Parsons Pantry. Abundant storage resides within the two large cabinets, each featuring adjustable full width shelves. Made of hardwoods and built to last, this pantry is a fine addition to any household.'}

In [99]:
rgx = re.compile('HOME/Furniture/Kitchen & Dining Furniture.*', re.IGNORECASE)
new_results = collection.find({'categoryPath': rgx},
                                    {'itemId': 1,
                                     'shortDescription' : 1,
                                     'imageEntities.thumbnailImage' : 1,
                                     'categoryNode' : 1,
                                     'categoryPath' : 1,
                                     'name' : 1})

In [100]:
new_results.count()

18891

In [102]:
cats = []
for result in new_results:
    cats.append(result.get('categoryPath',))

NameError: name 'urlopen' is not defined

In [49]:
len(set(cats))

95

That looks more reasonable. Lets try and count images

In [160]:

new_results = collection.find({'$and': [{'categoryPath': rgx}, {'imageEntities.thumbnailImage' : {'$exists': True}}]},
                                    {'itemId': 1,
                                     'categoryPath' : 1,
                                     'imageEntities.thumbnailImage' : 1
                                    })

In [161]:
new_results[4]

{'_id': ObjectId('5a514a0095cdbc13b4a6073e'),
 'categoryPath': 'Home/Furniture/Kitchen & Dining Furniture/Dining Chairs',
 'imageEntities': [{'thumbnailImage': 'https://i5.walmartimages.com/asr/eca96694-0796-4b72-8601-0eae4d378706_1.0c7bb1e7a070b537c3f1753af1671f6e.jpeg?odnHeight=100&odnWidth=100&odnBg=FFFFFF'},
  {'thumbnailImage': 'https://i5.walmartimages.com/asr/ea35dc0c-a985-469d-9af2-afacc48867b5_1.949c6f60e2fc225b1129277394e78a42.jpeg?odnHeight=100&odnWidth=100&odnBg=FFFFFF'},
  {'thumbnailImage': 'https://i5.walmartimages.com/asr/daef96e5-7db4-4fb8-a226-1dd24b73d270_1.4bc4ed8384196054524cc41ff59bc587.jpeg?odnHeight=100&odnWidth=100&odnBg=FFFFFF'},
  {'thumbnailImage': 'https://i5.walmartimages.com/asr/e43fc356-6c4d-43e4-b91d-db2844d95b64_1.d2e64d20c2f97bfb4e3b50453df403bb.jpeg?odnHeight=100&odnWidth=100&odnBg=FFFFFF'},
  {'thumbnailImage': 'https://i5.walmartimages.com/asr/649e65c8-aa4c-4a05-a3b0-520b1b0a45f6_1.894816805874f183bb0edeb3fb96ee59.jpeg?odnHeight=100&odnWidth=100&od

In [162]:
count = new_results.count()
count_num = 0
for i in range(0, 3001):
    if i % 500 == 0: print (i)
    count_num += len(new_results[i].get('imageEntities',[]))

0
500


KeyboardInterrupt: 

In [163]:
count_num/3001 * count

11239.051316227924

In [164]:
count

19373

In [165]:
img_num = 1
for i in range(100, 1000):
    if i % 100 == 0: print (i)
    item_id = new_results[i]['itemId']
    for item in new_results[i]['imageEntities']:
        time.sleep(0.2)
        url = item['thumbnailImage']
        
        filename = 'pics/{:06}_{}.jpg'.format(img_num,item_id)
        urllib.request.urlretrieve(url, filename)
        img_num += 1
    

0


FileNotFoundError: [Errno 2] No such file or directory: 'pics/Home/Furniture/Kitchen & Dining Furniture/Bar Stools/000001_10752302.jpg'

In [115]:
urllib.request.urlretrieve("https://i5.walmartimages.com/asr/b6090c4a-ffbe-44c1-99f0-065ebe4ea874_1.fd226a3e63ab622ebe447e78eeaae7e6.jpeg?odnHeight=100&odnWidth=100&odnBg=FFFFFF", "00000001.jpg")

('00000001.jpg', <http.client.HTTPMessage at 0x108f76b70>)

In [95]:
test[0]

TypeError: 'CommandCursor' object does not support indexing

In [142]:
new_results[10]

{'_id': ObjectId('5a514a0195cdbc13b4a6075a'),
 'imageEntities': [{'thumbnailImage': 'https://i5.walmartimages.com/asr/a0217b81-94fb-40cd-bf94-55c902e2c316_1.722c1e2ca1995143654aafa18f5c11bc.jpeg?odnHeight=100&odnWidth=100&odnBg=FFFFFF'},
  {'thumbnailImage': 'https://i5.walmartimages.com/asr/3aae2107-9ff1-45c5-bb81-ca6a9f4f1d5e_1.62697bfa0d7986f206b3f6e13b481cf2.jpeg?odnHeight=100&odnWidth=100&odnBg=FFFFFF'},
  {'thumbnailImage': 'https://i5.walmartimages.com/asr/5289dd13-5066-4cad-aef1-257372e21f37_1.5b5d7aa09f053bb65c35a9300aee8972.jpeg?odnHeight=100&odnWidth=100&odnBg=FFFFFF'},
  {'thumbnailImage': 'https://i5.walmartimages.com/asr/31635dd7-f516-4ac8-bead-ffc8fe3a3358_1.a258c0089daacfbd74cb87abbbd6963e.jpeg?odnHeight=100&odnWidth=100&odnBg=FFFFFF'}],
 'itemId': 13446521}

In [ ]:
csv.reader()